## MultiRC Demo

In this notebook, we describe how we can apply simple transformation functions to achieve state-of-the-art results on the **Multi-Sentence Reading Comprehension (MultiRC)** task.

In [1]:
import os
import random
import sys
from pathlib import Path

import pandas as pd
# Don't truncate the sentence when viewing examples
pd.set_option('display.max_colwidth', -1)

from snorkel.augmentation.apply import PandasTFApplier
from snorkel.augmentation.policy import RandomAugmentationPolicy
from snorkel.augmentation.tf import transformation_function

In [2]:
if not "cwd" in globals():
    cwd = Path(os.getcwd())
sys.path.insert(0, str(cwd.parents[0]))

## Loading data

Let's start by visualizing a few of the MultiRC examples.

Each example consists of a _paragraph_, a _question_, and a list of possible answers.
Each question can have multiple possible correct answers and might rely on context drawn from multiple sentences.

In [3]:
from dataloaders import get_jsonl_path
from superglue_parsers.multirc import get_rows

In [4]:
task_name = "MultiRC"
data_dir = os.environ["SUPERGLUEDATA"]
splits = ["train", "valid"]
max_data_samples = 1000 # note, we limit the number of examples to make the demo run faster :)

In [5]:
jsonl_paths = {}
multirc_dfs = {}

for split in splits:
    jsonl_paths[split] = get_jsonl_path(data_dir, task_name, split)
    multirc_dfs[split] = pd.DataFrame.from_records(get_rows(jsonl_paths[split], max_data_samples))

multirc_dfs["train"].head()

,aid,answer,label,paragraph,paragraph_sent_list,pid,qid,question,sent_used
0,0,Yes,True,"Animated history of the US. Of course the cartoon is highly oversimplified, and most critics consider it one of the weakest parts of the film. But it makes a valid claim which you ignore entirely: That the strategy to promote ""gun rights"" for white people and to outlaw gun possession by black people was a way to uphold racism without letting an openly terrorist organization like the KKK flourish. Did the 19th century NRA in the southern states promote gun rights for black people? I highly doubt it. But if they didn't, one of their functions was to continue the racism of the KKK. This is the key message of this part of the animation, which is again being ignored by its critics. Buell shooting in Flint. You write: ""Fact: The little boy was the class thug, already suspended from school for stabbing another kid with a pencil, and had fought with Kayla the day before"". This characterization of a six-year-old as a pencil-stabbing thug is exactly the kind of hysteria that Moore's film warns against. It is the typical right-wing reaction which looks for simple answers that do not contradict the Republican mindset. The kid was a little bastard, and the parents were involved in drugs -- case closed. But why do people deal with drugs? Because it's so much fun to do so? It is by now well documented that the CIA tolerated crack sales in US cities to fund the operation of South American ""contras"" It is equally well known that the so-called ""war on drugs"" begun under the Nixon administration is a failure which has cost hundreds of billions and made America the world leader in prison population (both in relative and absolute numbers).","[Animated history of the US., Of course the cartoon is highly oversimplified, and most critics consider it one of the weakest parts of the film., But it makes a valid claim which you ignore entirely: That the strategy to promote ""gun rights"" for white people and to outlaw gun possession by black people was a way to uphold racism without letting an openly terrorist organization like the KKK flourish., Did the 19th century NRA in the southern states promote gun rights for black people?, I highly doubt it., But if they didn't, one of their functions was to continue the racism of the KKK., This is the key message of this part of the animation, which is again being ignored by its critics., Buell shooting in Flint., You write: ""Fact: The little boy was the class thug, already suspended from school for stabbing another kid with a pencil, and had fought with Kayla the day before""., This characterization of a six-year-old as a pencil-stabbing thug is exactly the kind of hysteria that Moore's film warns against., It is the typical right-wing reaction which looks for simple answers that do not contradict the Republican mindset., The kid was a little bastard, and the parents were involved in drugs -- case closed., But why do people deal with drugs?, Because it's so much fun to do so?, It is by now well documented that the CIA tolerated crack sales in US cities to fund the operation of South American ""contras"" It is equally well known that the so-called ""war on drugs"" begun under the Nixon administration is a failure which has cost hundreds of billions and made America the world leader in prison population (both in relative and absolute numbers)., ]",0,0,Does the author claim the animated films message is that the NRA upholds racism?,"[1, 2, 3, 5]"
1,1,"Uphold, and continue",True,"Animated history of the US. Of course the cartoon is highly oversimplified, and most critics consider it one of the weakest parts of the film. But it makes a valid claim which you ignore entirely: That the strategy to promote ""gun rights"" for white people and to outlaw gun possession by black people was a way to uphold racism without letting an openly terrorist organization like the KKK flourish. Did the 19th century NRA in the southern stat

## Train vanilla model
To start, let's train a baseline with BERT to see how MultiRC performs.

In [6]:
bert_model = "bert-large-cased"
tokenizer_name = "bert-large-cased"
batch_size = 128
max_sequence_length = 256

In [7]:
import superglue_tasks

from snorkel.mtl.trainer import Trainer
from snorkel.mtl.snorkel_config import default_config as config

task = superglue_tasks.task_funcs[task_name](bert_model)

In [8]:
from snorkel.mtl.model import MultitaskModel

vanilla_model = MultitaskModel(
    name=f"SuperGLUE",
    tasks=[task], 
    dataparallel=False,
    device=0 # use CPU
)

In [9]:
from snorkel.mtl.data import MultitaskDataLoader

from dataloaders import get_dataloaders
from superglue_parsers.multirc import parse_from_rows
from tokenizer import get_tokenizer

tokenizer = get_tokenizer(tokenizer_name)

# Construct training dataloader from original DF
dataloaders = []

for split in splits:
    rows = multirc_dfs[split].to_dict("records")
    dataset = parse_from_rows(rows, tokenizer, max_sequence_length)
    dataloaders.append(MultitaskDataLoader(
        task_to_label_dict={task_name: "labels"},
        dataset=dataset,
        split=split,
        batch_size=batch_size,
        shuffle=True if split == 'train' else False,
    ))

In [10]:
trainer_config = {
    "lr": 1e-5,
    "optimizer": "adam",
    "n_epochs": 10,
    "conter_unit": "epochs",
    "evaluation_freq": 0.25,
    "grad_clip": 5.0,
    "amsgrad": True,
}

# 1. Train your own model
# trainer = Trainer(**trainer_config)
# trainer.train_model(vanilla_model, dataloaders)

# 2. Use our pretrained model!
# multirc_vanilla_path = "TODO"
# vanilla_model.load(multirc_vanilla_path)

In [11]:
%%time
vanilla_model.score(dataloaders[1])

CPU times: user 29.7 s, sys: 6.52 s, total: 36.2 s
Wall time: 33.9 s


{'MultiRC/SuperGLUE/valid/f1': 0.3844086021505376}

## Write transformation functions

We notice a simple pattern over the original data— _question answers often rely on a subset of the context paragraphs—provided by metadata!_

Let's write a transformation function (TF) to reduce the length of input that our model will need to encode.

In [12]:
@transformation_function
def reduce_para_length(x):
    # Only use the relevant sentences 
    if len(x.sent_used) > 0:
        x.paragraph = " ".join([x.paragraph_sent_list[i] for i in x.sent_used])
    return x

In this case, we trivially apply this TF to every example in the dataset.

In [13]:
tfs = [reduce_para_length]
policy = RandomAugmentationPolicy(len([reduce_para_length]), sequence_length=1)

Note, that we set `keep_original=False`, which means we _do not want to keep the original examples_.
In other words, we are replacing the longer paragraphs with reduced versions!

In [14]:
random.seed(1)
multirc_dfs_reduced = {}
applier = PandasTFApplier(tfs, policy, k=1, keep_original=False)

for split in splits:
    multirc_dfs_reduced[split] = applier.apply(multirc_dfs[split])

100%|██████████| 1000/1000 [00:00<00:00, 1646.93it/s]


Let's take a look at some of the reduced examples.

In [15]:
multirc_dfs_reduced['train'].head()

,aid,answer,label,paragraph,paragraph_sent_list,pid,qid,question,sent_used
0,0,Yes,True,"Of course the cartoon is highly oversimplified, and most critics consider it one of the weakest parts of the film. But it makes a valid claim which you ignore entirely: That the strategy to promote ""gun rights"" for white people and to outlaw gun possession by black people was a way to uphold racism without letting an openly terrorist organization like the KKK flourish. Did the 19th century NRA in the southern states promote gun rights for black people? But if they didn't, one of their functions was to continue the racism of the KKK.","[Animated history of the US., Of course the cartoon is highly oversimplified, and most critics consider it one of the weakest parts of the film., But it makes a valid claim which you ignore entirely: That the strategy to promote ""gun rights"" for white people and to outlaw gun possession by black people was a way to uphold racism without letting an openly terrorist organization like the KKK flourish., Did the 19th century NRA in the southern states promote gun rights for black people?, I highly doubt it., But if they didn't, one of their functions was to continue the racism of the KKK., This is the key message of this part of the animation, which is again being ignored by its critics., Buell shooting in Flint., You write: ""Fact: The little boy was the class thug, already suspended from school for stabbing another kid with a pencil, and had fought with Kayla the day before""., This characterization of a six-year-old as a pencil-stabbing thug is exactly the kind of hysteria that Moore's film warns against., It is the typical right-wing reaction which looks for simple answers that do not contradict the Republican mindset., The kid was a little bastard, and the parents were involved in drugs -- case closed., But why do people deal with drugs?, Because it's so much fun to do so?, It is by now well documented that the CIA tolerated crack sales in US cities to fund the operation of South American ""contras"" It is equally well known that the so-called ""war on drugs"" begun under the Nixon administration is a failure which has cost hundreds of billions and made America the world leader in prison population (both in relative and absolute numbers)., ]",0,0,Does the author claim the animated films message is that the NRA upholds racism?,"[1, 2, 3, 5]"
1,1,"Uphold, and continue",True,"Of course the cartoon is highly oversimplified, and most critics consider it one of the weakest parts of the film. But it makes a valid claim which you ignore entirely: That the strategy to promote ""gun rights"" for white people and to outlaw gun possession by black people was a way to uphold racism without letting an openly terrorist organization like the KKK flourish. Did the 19th century NRA in the southern states promote gun rights for black people? But if they didn't, one of their functions was to continue the racism of the KKK.","[Animated history of the US., Of course the cartoon is highly oversimplified, and most critics consider it one of the weakest parts of the film., But it makes a valid claim which you ignore entirely: That the strategy to promote ""gun rights"" for white people and to outlaw gun possession by black people was a way to uphold racism without letting an openly terrorist organization like the KKK flourish., Did the 19th century NRA in the southern states promote gun rights for black people?, I highly doubt it., But if they didn't, one of their functions was to continue the racism of the KKK., This is the key message of this part of the animation, which is again being ignored by its critics., Buell shooting in Flint., You write: ""Fact: The little boy was the class thug, already suspended from school for stabbing another kid with a pencil, and had fought with Kayla the day before""., This characterization of a six-year-old as a pencil-stabbing thug is exactly the kind of hysteria that Moore's film warns again

## Train with augmented data

Now, let's train a MultiRC model with the transformed data.

In [16]:
augmented_model = MultitaskModel(
    name=f"SuperGLUE",
    tasks=[task], 
    dataparallel=False,
    device=0
)

In [17]:
# Construct training dataloader from augmented DF
dataloaders = []

for split in splits:
    rows = multirc_dfs_reduced[split].to_dict("records")
    dataset = parse_from_rows(rows, tokenizer, max_sequence_length)
    dataloaders.append(MultitaskDataLoader(
        task_to_label_dict={task_name: "labels"},
        dataset=dataset,
        split=split,
        batch_size=batch_size,
        shuffle=True if split == 'train' else False,
    ))

In [18]:
# 1. Train your own model
# trainer = Trainer(**trainer_config)
# trainer.train_model(augmented_model, dataloaders)

# 2. Use our pretrained model!
multirc_slice_path = "/dfs/scratch1/senwu/mmtl/snorkel-superglue/notebooks/my_best_multirc_tf_model.pth"
augmented_model.load(multirc_slice_path)

In [19]:
%%time
augmented_model.score(dataloaders[1])

CPU times: user 14.2 s, sys: 3.48 s, total: 17.7 s
Wall time: 17.6 s


{'MultiRC/SuperGLUE/valid/f1': 0.7098901098901099}